In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.13.0'

## 1. Preprocessing

### 1.1. Load and check descriptive stats of the dataset

In [3]:
data=pd.read_csv(r"C:\Users\Administrator\Downloads\Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


### 1.2. Splitting X and Y

In [6]:
X=data.iloc[:,3:-1].values
Y=data.iloc[:,-1].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

- ignore RowNumber,Customer Id, Surname
- we can further one hot-encode Geography and Gender as 0 and 1
- why not LabelEncoding for Geography? remember that if the 3 categories are related, then we can do LabelEncoding, otherwise prefer one hot encoding for >2 labels

In [7]:
Y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [8]:
print(X.shape,Y.shape)

(10000, 10) (10000,)


### 1.3. Label Encoding and One Hot Encoding

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

In [11]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [13]:
ct=ColumnTransformer(transformers=[('encoder' , OneHotEncoder(), [1])],remainder='passthrough')# the 1 represents which column to one hot encode
X=np.array(ct.fit_transform(X))

In [14]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

- note that the one hot-encoded features shift to first few columns
- France- 1,0,0; Germany- 0,1,0; Spain- 0,0,1

## 2. Split into Train and Test Set

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

### Feature Scaling is very Important for Neural Networks

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train) #note the difference, here we have to fit and transform
X_test=sc.transform(X_test)

## 3. Building the ANN
- the neural network is basically an object of the Sequential Class, which allows us to create a NN in a sequence format to enable easier synaptic connections, layer by layer
- Boltzmann machines actually have random connections of neurons in a computational graph format

### 3.1. Instantiate the neural networks and initialse the parameters

In [19]:
ann=tf.keras.models.Sequential()

- in the Dense object, we have different parameters, most imp ones being units, activation, kernel_regularizer
- we also use Dense layers for hidden layers only
- no rule of thumb for fixed number of neurons/layers

In [20]:
ann.add(tf.keras.layers.Dense(units=16,activation='relu')) #1st hidden layer
ann.add(tf.keras.layers.Dense(units=8,activation='relu')) #2nd hidden layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid')) #Output hidden layer

## 4. Training the ANN

### 4.1. Compiling ANN using optimizer, error metric, loss function

- Adam optimizer for SGD if binary classes, SoftMax for multpile classes
- loss to choose would be binary_crossentropy for binary classes, catagorical_crossentropy for multiple classes

In [21]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### 4.2 Actual training

- Mini Batch GD

In [22]:
ann.fit(X_train,Y_train, batch_size=16, epochs=50) #batch_size is usually fixed

Epoch 1/50
500/500 [==============================] - 1s 1ms/step - loss: 0.4860 - accuracy: 0.7805
Epoch 2/50
500/500 [==============================] - 1s 2ms/step - loss: 0.4259 - accuracy: 0.8080
Epoch 3/50
500/500 [==============================] - 1s 2ms/step - loss: 0.4088 - accuracy: 0.8255
Epoch 4/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3922 - accuracy: 0.8365
Epoch 5/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3702 - accuracy: 0.8489
Epoch 6/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3529 - accuracy: 0.8569
Epoch 7/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3435 - accuracy: 0.8591
Epoch 8/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3390 - accuracy: 0.8604
Epoch 9/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3376 - accuracy: 0.8586
Epoch 10/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3359 - accuracy: 0.8604

## 5. Predicting For a New Customer

- while predicting, note 3 things
    - 2D array
    - maintain same encoding
    - maintain same scaling

In [23]:
threshold=0.5

In [24]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000,]])) > threshold) #input should be 2D array, training/testing/predicting 

1/1 [==============================] - 0s 221ms/step
[[False]]


## 6. Evaluation
- Confusion Matrix

In [25]:
Y_pred=ann.predict(X_test)
Y_pred=(Y_pred>0.5)

63/63 [==============================] - 0s 2ms/step


In [26]:
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(Y_test,Y_pred)
print(cm)
accuracy_score(Y_test,Y_pred)

[[1516   79]
 [ 201  204]]


0.86

### ANN Done!!